In [2]:
import pandas as pd
import numpy as np
import math
import tensorflow as tf
from sklearn.utils import shuffle
from transformers import T5TokenizerFast, TFT5ForConditionalGeneration
from progressbar import progressbar as pb


In [3]:
data = pd.read_csv('new_dataset.csv',index_col=0)
data=data[['txt','PERSON','NORP','JOB','JOB1','GPE','ORG','TENSE']]
data['TENSE'] = np.where((data['TENSE'] == 'was'),'PAST','PRESENT')
data=shuffle(data)

n = 30000
df_train = data.iloc[:n,1:]
df_label = data.iloc[:n,0:1]


In [4]:
model = TFT5ForConditionalGeneration.from_pretrained("t5-small")
tokenizer = T5TokenizerFast.from_pretrained("t5-small")
optimizer = tf.keras.optimizers.Adam()

2021-07-13 19:41:54.879000: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-07-13 19:41:54.879282: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-07-13 19:41:54.879416: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (HAL-9000): /proc/driver/nvidia/version does not exist
2021-07-13 19:41:54.882593: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-07-13 19:41:54.949502: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but

In [5]:
batch_size=32
num_of_batches=math.floor(len(df_train)/batch_size)

In [6]:
label=[df_label.values[i][0] for i in range (len(df_label))]
label_encoded=tokenizer.batch_encode_plus(label,padding=True,max_length=900,return_tensors='tf')["input_ids"]

inputbatch=['|'.join(df_train.iloc[i].dropna().values.tolist()) for i in range (len(df_train))]
input_encoded=tokenizer.batch_encode_plus(inputbatch,padding=True,max_length=900,return_tensors='tf')["input_ids"]

In [7]:
epoch=1
for k in range(epoch):

    running_loss=0
    
    
    for i in pb(range(num_of_batches)):
        input=input_encoded[i*batch_size:i*batch_size+batch_size]
        output=label_encoded[i*batch_size:i*batch_size+batch_size]
        
        #Entrainement sur le batch
        with tf.GradientTape() as tape:
            outputs = model(input_ids=input, labels=output)
            # Compute the loss value for this batch.
            loss_value = sum(outputs.loss)/len(outputs.loss)
            running_loss+=loss_value
        # Actualisation des poids
        gradients = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(gradients, model.trainable_weights))



N/A% (0 of 3) |                          | Elapsed Time: 0:00:00 ETA:  --:--:--
2021-07-13 19:30:10.281597: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 168607744 exceeds 10% of free system memory.
2021-07-13 19:30:11.903808: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 168607744 exceeds 10% of free system memory.
2021-07-13 19:30:12.404566: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 168607744 exceeds 10% of free system memory.
2021-07-13 19:30:14.384396: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 168607744 exceeds 10% of free system memory.
2021-07-13 19:30:14.858102: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 168607744 exceeds 10% of free system memory.
 33% (1 of 3) |########                  | Elapsed Time: 0:00:15 ETA:   0:00:31
 66% (2 of 3) |#################         | Elapsed Time: 0:00:30 ETA:   0:00:14
100% (3 of 3) |##########################| Elapsed Time

In [9]:
model.save_pretrained("../model")